# Laba 3

In [1]:
import numpy as np
from graphblas import Matrix, Vector
from graphblas import dtypes
from graphblas import binary, semiring

# #1 Bellman–Ford (1 стартовая вершина)

In [2]:
def Bellman_Ford(A, vertex):
    
    m = A.nrows
    v = Vector(A.dtype, m)
    v[vertex] << 0
    
    w = v.dup()
    for i in range(m - 1):
        w_old = w.dup()
        w(binary.min) << w.vxm(A, semiring.min_plus)
        if w.isequal(w_old):
            break
            
    w_old = w.dup()
    w(binary.min) << w.vxm(A, semiring.min_plus)
    if not w.isequal(w_old):
        raise RuntimeError('Цикл отрицательного веса')
 
    final = []
    for i in range(m):
        if w[i].value is None:
            final.append(float('inf'))
        else:
            final.append(w[i].value)
    return final

# #2 Bellman–Ford (несколько стартовых вершин)

In [3]:
def Bellman_Ford2(A, massiv):
    
    n = A.ncols
    m = massiv.size
    
    v = Matrix(dtypes.INT32, m, n)
    for i in range(m):
        v[i, massiv[i]] << 0
        
    w = v.dup()
    for i in range(n - 1):
        w_old = w.dup()
        for i in range(m):
            w[i, :](binary.min) << w[i, :].vxm(A, semiring.min_plus)
        if w.isequal(w_old):
            break
    
    w_old = w.dup()
    w(binary.min) << w.vxm(A, semiring.min_plus)
    if not w.isequal(w_old):
        raise RuntimeError('Цикл отрицательного веса')

    final = {}
    for i in range(m):
        values = []
        for j in range(n):
            if w[i, j].value is None: 
                values.append(float('inf'))
            else:
                values.append(w[i, j].value)
        final[massiv[i]] = (values)
    return final

# #3 Floyd–Warshall

In [4]:
def FloydWarshall(matrix):
    
    dist = matrix.dup()
    n = matrix.nrows
    temp_dist = matrix.dup()
    
    for k in range(n):
        
        dist_col = Matrix(dtypes.INT32, n, n)
        dist_row = Matrix(dtypes.INT32, n, n)
        dist_col[:, k] = dist[:, k].dup()
        dist_row[k, :] = dist[k, :].dup()
        temp_dist << dist_col.mxm(dist_row, semiring.min_plus)

        dist << dist.ewise_add(temp_dist, op="min")
        
    dist1 = dist.dup()
    temp_dist1 = temp_dist.dup()
    dist_col1 = Matrix(dtypes.INT32, n, n)
    dist_row1 = Matrix(dtypes.INT32, n, n)
    dist_col1[:, k] = dist1[:, k].dup()
    dist_row1[k, :] = dist1[k, :].dup()
    temp_dist1 << dist_col1.mxm(dist_row1, semiring.min_plus)
    dist1 << dist1.ewise_add(temp_dist1, op="min")
    
    if not dist1.isequal(dist):
        raise RuntimeError('Цикл отрицательного веса')

    result = {}
    for i in range(n):
        val_list = []
        for j in range(n):
            if i == j:
                val_list.append(0)
            elif dist[i, j].value is None:
                val_list.append(float('inf'))
            else:
                val_list.append(dist[i, j].value)
        result[i] = np.array(val_list)
    return result


# #3 Вычисление транзитивного замыкания

In [5]:
def TransitiveClosure(m):
    
    rows = m.nrows
    I = Matrix(dtypes.INT32, rows, rows)
    
    for i in range(rows):
        I[i, i] = 1
    D = I.dup()
    
    for i in range(rows):
        D1 = D.dup()
        D = D.mxm(m, semiring.min_plus).ewise_add(I, op="min")
        if D1.isequal(D):
            break
    
    D2 = D.dup()
    D2 = D2.mxm(m, semiring.min_plus).ewise_add(I, op="min")

    if not D2.isequal(D):
        raise RuntimeError('Цикл отрицательного веса')

    output = {}
    
    for i in range(rows):
        ar_val = []
        for j in range(rows):
            if D[i, j].value is None:
                ar_val.append(float('inf'))
            elif i == j:
                ar_val.append(0)
            else:
                ar_val.append(D[i, j].value - 1)
        output[i] = np.array(ar_val)
    return output